In [1]:
import glob
from model_RA.utils.dataloader_keras import genUnbalSequence
import os
import sys
import yaml
import numpy as np
from eval_RA.utils.get_index_faiss import get_index
from eval_RA.utils.print_table import PrintTable
import time
from model_RA.fp_RA.melspec.melspectrogram_RA import get_melspec_layer
import tensorflow as tf
from model_RA.fp_RA.nnfp import get_fingerprinter

2024-06-15 20:55:33.883188: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [3]:
config = "default_RA"
cfg = load_config(config)

cli: Configuration from ./config/default_RA.yaml


In [4]:
# Data location
#source_root_dir = '/mnt/dataset/public/Fingerprinting/query_procura'
source_root_dir = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000'

# Source (music) file paths
ts_query_icassp_fps = sorted(
                glob.glob(source_root_dir + '/000002.wav', recursive=True)) #'/*.wav'

# BSZ
ts_batch_sz = cfg['BSZ']['TS_BATCH_SZ']

# Model parameters
dur = cfg['MODEL']['DUR']
hop = cfg['MODEL']['HOP']
fs = cfg['MODEL']['FS']

m_pre = get_melspec_layer(cfg, trainable=False)
m_fp = get_fingerprinter(cfg, trainable=False)

In [5]:
ts_query_icassp_fps

['/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/train-10k-30s/fma_small_8k_plus_medium_2k/000/000002.wav']

In [6]:
_ts_n_anchor = ts_batch_sz
ds_query = genUnbalSequence(
    ts_query_icassp_fps,
    ts_batch_sz,
    _ts_n_anchor,
    dur,
    hop,
    fs,
    shuffle=False,
    random_offset_anchor=False,
    drop_the_last_non_full_batch=False) # No augmentations...

In [7]:
ds = dict()
ds['query_1'] = ds_query

In [8]:
"""
enq = tf.keras.utils.OrderedEnqueuer(
            ds_query, use_multiprocessing=True, shuffle=False)
enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],
            max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])

#i = 0
#while i < len(enq.sequence)
X = next(enq.get()) #(59, 1, 8000) + (0, 1, 8000)
X = tf.concat(X, axis=0) #TensorShape([59, 1, 8000])
feat = m_pre(X)  # (nA+nP, F, T, 1) #<tf.Tensor: shape=(59, 256, 32, 1), dtype=float32,
#enq.stop()
"""

"\nenq = tf.keras.utils.OrderedEnqueuer(\n            ds_query, use_multiprocessing=True, shuffle=False)\nenq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],\n            max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])\n\n#i = 0\n#while i < len(enq.sequence)\nX = next(enq.get()) #(59, 1, 8000) + (0, 1, 8000)\nX = tf.concat(X, axis=0) #TensorShape([59, 1, 8000])\nfeat = m_pre(X)  # (nA+nP, F, T, 1) #<tf.Tensor: shape=(59, 256, 32, 1), dtype=float32,\n#enq.stop()\n"

In [9]:
#m_fp(m_pre(X)) 

In [10]:
def emb_step(X, m_pre, m_fp):
    m_fp.trainable = False
    #emb_f = m_fp.front_conv(feat)  # (BSZ, Dim) #<tf.Tensor: shape=(59, 1024),W
    #emb_gf = m_fp.div_enc(emb_f)
    #emb_gf = tf.math.l2_normalize(emb_gf, axis=1) #<tf.Tensor: shape=(59, 128), dtype=float32,
    #m_fp(m_pre(X)) 
    return m_fp(m_pre(X))#emb_gf

In [11]:
enq = tf.keras.utils.OrderedEnqueuer(ds_query,
                                        use_multiprocessing=True,
                                        shuffle=False)
enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],
            max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])

In [12]:
X, _ = next(enq.get())
#X = tf.concat(X, axis=0)

In [14]:
X.shape

(58, 1, 8000)

In [12]:
output_root_dir = '/mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/query-134/'

sz_check = dict() # for warning message
for key in ds.keys():
    bsz = int(cfg['BSZ']['TS_BATCH_SZ'])  # Do not use ds.bsz here.
    # n_items = len(ds[key]) * bsz
    n_items = ds[key].n_samples
    dim = cfg['MODEL']['EMB_SZ']
    #print(key, bsz, n_items, dim)

    # Create memmap, and save shapes
    assert n_items > 0
    arr_shape = (n_items, dim)
    arr = np.memmap(f'{output_root_dir}/{key}.mm',
                    dtype='float32',
                    mode='w+',
                    shape=arr_shape)
    np.save(f'{output_root_dir}/{key}_shape.npy', arr_shape)

    # Fingerprinting loop
    enq = tf.keras.utils.OrderedEnqueuer(ds[key],
                                            use_multiprocessing=True,
                                            shuffle=False)
    enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],
                max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])
    i = 0
    while i < len(enq.sequence):
        X, _ = next(enq.get())
        X = tf.concat(X, axis=0)
        emb = emb_step(X, m_pre, m_fp)
        arr[i * bsz:(i + 1) * bsz, :] = emb.numpy() # Writing on disk.
        i += 1
    enq.stop()

    sz_check[key] = len(arr)
    #print(len(arr))
    arr.flush(); del(arr) # Close memmap

In [13]:
def load_memmap_data(source_dir,
                     fname,
                     append_extra_length=None,
                     shape_only=False,
                     display=True):

    path_shape = source_dir + fname + '_shape.npy'
    path_data = source_dir + fname + '.mm'
    data_shape = np.load(path_shape)
    if shape_only:
        return data_shape

    if append_extra_length:
        data_shape[0] += append_extra_length
        data = np.memmap(path_data, dtype='float32', mode='r+',
                         shape=(data_shape[0], data_shape[1]))
    else:
        data = np.memmap(path_data, dtype='float32', mode='r',
                         shape=(data_shape[0], data_shape[1]))
    if display:
        print(f'Load {data_shape[0]:,} items from \033[32m{path_data}\033[0m.')
    return data, data_shape

In [14]:
logsDir = '/mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101'
emb_dir = logsDir + '/'
emb_dummy_dir = None
index_type='ivfpq'
nogpu=False
max_train=1e7
test_ids='icassp'
test_seq_len='1'
k_probe=20
display_interval=5

In [15]:
test_seq_len='1'

test_seq_len = np.asarray(
        list(map(int, test_seq_len.split())))  # '1 3 5' --> [1, 3, 5]

query, query_shape = load_memmap_data(emb_dir, 'query')
db, db_shape = load_memmap_data(emb_dir, 'db')

if emb_dummy_dir is None:
    emb_dummy_dir = emb_dir

dummy_db, dummy_db_shape = load_memmap_data(emb_dummy_dir, 'dummy_db')

Load 29,500 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/query.mm.
Load 29,500 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/db.mm.
Load 53,754,198 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/dummy_db.mm.


In [16]:
query_dir = '/mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/query-134/'
query_1, query_1_shape = load_memmap_data(query_dir, 'query_1')

Load 59 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/query-134/query_1.mm.


In [18]:
import faiss
d = dummy_db.shape[1]
indice = faiss.IndexFlatL2(d)


In [1]:
code_sz = 64 # power of 2
n_centroids = 256#
nbits = 8  # nbits must be 8, 12 or 16, The dimension d should be a multiple of M.
index = faiss.IndexIVFPQ(indice, d, n_centroids, code_sz, nbits)

NameError: name 'faiss' is not defined

In [20]:
code_sz = 64 # power of 2
n_centroids = 256#
nbits = 8  # nbits must be 8, 12 or 16, The dimension d should be a multiple of M.
index = faiss.IndexIVFPQ(indice, d, n_centroids, code_sz, nbits)

# Se não usar GPU
if not nogpu:
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)

: 

In [19]:
if not index.is_trained:
    index.train(db_embeddings)

True

In [18]:
index = get_index(index_type, dummy_db, dummy_db.shape, (not nogpu),
                max_train)

# Add items to index
start_time = time.time()

index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')
#index.add(db); print(f'{len(db)} items from reference DB')
index.add(query_1); print(f'{len(query_1)} items from dummy DB')

t = time.time() - start_time
print(f'Added total {index.ntotal} items to DB. {t:>4.2f} sec.')

: 

In [1]:
import glob
import os
# Carregar os nomes das músicas da base de dados
_prefix = 'train-10k-30s/'
source_root_dir = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/'
music_files = sorted(glob.glob(source_root_dir + _prefix + '**/*.wav',
                      recursive=True))
#print(music_files)
#music_files = sorted(glob.glob(source_root_dir + '**/*.wav'))
music_names = [os.path.basename(music_file) for music_file in music_files]
print(music_names)

['000002.wav', '000005.wav', '000010.wav', '000140.wav', '000141.wav', '000148.wav', '000182.wav', '000190.wav', '000193.wav', '000194.wav', '000197.wav', '000200.wav', '000203.wav', '000204.wav', '000207.wav', '000210.wav', '000211.wav', '000212.wav', '000213.wav', '000255.wav', '000256.wav', '000352.wav', '000368.wav', '000400.wav', '000424.wav', '000459.wav', '000534.wav', '000540.wav', '000546.wav', '000574.wav', '000583.wav', '000602.wav', '000603.wav', '000615.wav', '000620.wav', '000621.wav', '000625.wav', '000666.wav', '000667.wav', '000676.wav', '000680.wav', '000690.wav', '000694.wav', '000695.wav', '000703.wav', '000704.wav', '000705.wav', '000706.wav', '000707.wav', '000708.wav', '000709.wav', '000714.wav', '000715.wav', '000716.wav', '000718.wav', '000745.wav', '000750.wav', '000777.wav', '000814.wav', '000821.wav', '000822.wav', '000825.wav', '000830.wav', '000853.wav', '000863.wav', '000881.wav', '000890.wav', '000892.wav', '000897.wav', '000992.wav', '000993.wav', '0009